In [1]:
pip install dask[dataframe]

Defaulting to user installation because normal site-packages is not writeable
     -------------------------------------- 241.9/241.9 kB 7.2 MB/s eta 0:00:00
     ------------------------------------- 242.8/242.8 kB 14.5 MB/s eta 0:00:00
     ------------------------------------- 241.7/241.7 kB 14.5 MB/s eta 0:00:00
     ------------------------------------- 206.6/206.6 kB 13.1 MB/s eta 0:00:00
     ------------------------------------- 205.9/205.9 kB 13.0 MB/s eta 0:00:00
     ------------------------------------- 205.9/205.9 kB 13.0 MB/s eta 0:00:00
     ------------------------------------- 205.7/205.7 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.5/205.5 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.4/205.4 kB 12.2 MB/s eta 0:00:00
     ------------------------------------- 205.1/205.1 kB 12.2 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
import pandas as pd
import numpy as np
pd.plotting.register_matplotlib_converters()
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeRegressor
print("Vis setup Complete")
print("Setup complete.")

Vis setup Complete
Setup complete.


In [2]:
#Load the data using pandas read_csv function. This will generate a data frame we will use through the work.
cleaned_goalkeepers_df = pd.read_csv('goalkeepers_rating_2.csv')
cleaned_midfielders_df = pd.read_csv('midfielders_rating_2.csv')
cleaned_forwarders_df = pd.read_csv('forwarders_rating_2.csv')
cleaned_defenders_df = pd.read_csv('defenders_rating_2.csv')

In [3]:
#Show the first and last few records to check data is loading correctly
print(cleaned_defenders_df.head())
print(cleaned_defenders_df.tail())

  competition        date pos_role          player  goals  assists  \
0   Euro 2016  2016-06-10       DC  Dragos Grigore      0        0   
1   Euro 2016  2016-06-10       DC  Dragos Grigore      0        0   
2   Euro 2016  2016-06-10       DC  Dragos Grigore      0        0   
3   Euro 2016  2016-06-10       DL      Razvan Rat      0        0   
4   Euro 2016  2016-06-10       DL      Razvan Rat      0        0   

   shots_ontarget  shots_offtarget  shotsblocked  chances2score  ...  \
0               0                0             0              0  ...   
1               0                0             0              0  ...   
2               0                0             0              0  ...   
3               0                1             1              0  ...   
4               0                1             1              0  ...   

   missed_penalties  owngoals  degree_centrality  betweenness_centrality  \
0                 0         0           1.071429                0.1430

In [4]:
#shape
cleaned_defenders_df.shape

(14945, 54)

In [5]:
#check the single values of the columns

In [6]:
import pandas as pd

df = cleaned_defenders_df

# Iterate through each column and check for columns with a single unique value
single_value_columns = [column for column in df.columns if df[column].nunique() == 1]

# Print columns with a single unique value
print("Columns with a single unique value:")
for column in single_value_columns:
    print(column)

Columns with a single unique value:
goals_ag_otb
goals_ag_itb
saves_itb
saves_otb
saved_pen


In [7]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['goals_ag_otb',
'goals_ag_itb',
'saves_itb',
'saves_otb',                   
'saved_pen']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_defenders_df = cleaned_defenders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_defenders_df.head()

,competition,date,pos_role,player,goals,assists,shots_ontarget,shots_offtarget,shotsblocked,chances2score,...,missed_penalties,owngoals,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,flow_success,betweenness2goals,minutesPlayed,game_duration
0,Euro 2016,2016-06-10,DC,Dragos Grigore,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.0,0.0,90,90
1,Euro 2016,2016-06-10,DC,Dragos Grigore,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.0,0.0,90,90
2,Euro 2016,2016-06-10,DC,Dragos Grigore,0,0,0,0,0,0,...,0,0,1.071429,0.143055,0.603571,0.304348,0.0,0.0,90,90
3,Euro 2016,2016-06-10,DL,Razvan Rat,0,0,0,1,1,0,...,0,0,1.500000,0.321679,0.754464,0.478261,0.0,0.0,90,90
4,Euro 2016,2016-06-10,DL,Razvan Rat,0,0,0,1,1,0,...,0,0,1.500000,0.321679,0.754464,0.478261,0.0,0.0,90,90


In [8]:
#number of players in each competition
# Group by competition and count the number of unique players in each competition
player_count_per_competition = cleaned_defenders_df.groupby('competition')['player'].nunique().reset_index(name='unique_player_count')

# Display the results
player_count_per_competition

,competition,unique_player_count
0,Bundesliga 2017-18,163
1,Euro 2016,127
2,Premier League 2017-18,182
3,World Cup 2018,181


In [9]:
# Determine the first pos_role for each player in each competition
first_pos_role = cleaned_defenders_df.groupby(['competition', 'player'])['pos_role'].first().reset_index()

# Merge this first pos_role back into the original dataframe
cleaned_defenders_df = cleaned_defenders_df.drop(columns=['pos_role'])
cleaned_defenders_df = cleaned_defenders_df.merge(first_pos_role, on=['competition', 'player'], how='left')

# Calculate the number of matches played per player in each competition
cleaned_defenders_df['matches_played'] = cleaned_defenders_df.groupby(['competition', 'player'])['date'].transform('nunique')

# Calculate the average network metrics, game duration, and minutes played
network_metrics = ['degree_centrality', 'betweenness_centrality', 'closeness_centrality', 'flow_centrality',  'betweenness2goals', 'game_duration','flow_success','minutesPlayed']
cleaned_defenders_df[network_metrics] = cleaned_defenders_df.groupby(['competition', 'player'])[network_metrics].transform('mean')

# Select only numeric columns for summing
columns_to_sum = cleaned_defenders_df.select_dtypes(include='number').columns.difference(['matches_played'] + network_metrics)
cleaned_defenders_df[columns_to_sum] = cleaned_defenders_df.groupby(['competition', 'player'])[columns_to_sum].transform('sum')

# Drop duplicate rows and the date column
cleaned_defenders_df = cleaned_defenders_df.drop_duplicates(subset=['competition', 'player'])
cleaned_defenders_df = cleaned_defenders_df.drop(columns=['date'])

# Reorder columns to maintain the original order
ordered_columns = ['competition', 'pos_role', 'player', 'matches_played'] + list(columns_to_sum) + network_metrics
cleaned_defenders_df = cleaned_defenders_df[ordered_columns]

In [10]:
cleaned_defenders_df.head()

,competition,pos_role,player,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,Euro 2016,DC,Dragos Grigore,3,17,20,0,0,60,14,...,16,4,1.123377,0.181910,0.622351,0.333472,0.0,90.0,0.490429,90.000000
3,Euro 2016,DL,Razvan Rat,2,11,12,0,0,19,3,...,3,3,1.214286,0.232282,0.729104,0.345320,0.0,90.0,0.310037,74.000000
6,Euro 2016,DC,Laurent Koscielny,6,20,101,0,15,75,11,...,21,4,1.222135,0.145895,0.647770,0.362843,0.0,90.0,2.182100,87.428571
9,Euro 2016,DC,Vlad Chiriches,3,4,3,0,0,30,3,...,12,3,1.175325,0.202411,0.640516,0.380556,0.0,90.0,0.761140,90.000000
12,Euro 2016,DR,Cristian Sapunaru,3,19,24,0,4,30,0,...,20,4,1.207792,0.206028,0.666690,0.313085,0.0,90.0,1.615328,90.000000


In [11]:
#to check we have some players that played in other competitions
import pandas as pd


# Function to count the number of competitions for each player
def count_competitions_per_player(dataframe):
    return dataframe.groupby('player')['competition'].nunique().reset_index(name='Number of Competitions')

# Calculate the number of competitions for each player
player_competitions = count_competitions_per_player(cleaned_defenders_df)

# Output the result
print(player_competitions)


                player  Number of Competitions
0      Aaron Cresswell                       1
1         Aaron Hughes                       1
2    Aaron Wan-Bissaka                       1
3         Abdou Diallo                       1
4      Abdullahi Shehu                       1
..                 ...                     ...
545    Youssouf Sabaly                       1
546      Yun Young-Sun                       1
547       Yuri Zhirkov                       1
548      Yuto Nagatomo                       1
549              Zanka                       2

[550 rows x 2 columns]


In [12]:
#we dropped the columns, that has single value in the dataset and it is known that all goal leepers here played for 90 minutes as well

columns_to_drop = ['competition','player']

# Dropping the columns and retaining the name goalkeepers_df
cleaned_defenders_df = cleaned_defenders_df.drop(columns=columns_to_drop)

# Display the first few rows of the cleaned dataframe to verify
cleaned_defenders_df.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,DC,3,17,20,0,0,60,14,0,0,...,16,4,1.123377,0.181910,0.622351,0.333472,0.0,90.0,0.490429,90.000000
3,DL,2,11,12,0,0,19,3,13,4,...,3,3,1.214286,0.232282,0.729104,0.345320,0.0,90.0,0.310037,74.000000
6,DC,6,20,101,0,15,75,11,0,0,...,21,4,1.222135,0.145895,0.647770,0.362843,0.0,90.0,2.182100,87.428571
9,DC,3,4,3,0,0,30,3,0,0,...,12,3,1.175325,0.202411,0.640516,0.380556,0.0,90.0,0.761140,90.000000
12,DR,3,19,24,0,4,30,0,0,20,...,20,4,1.207792,0.206028,0.666690,0.313085,0.0,90.0,1.615328,90.000000


In [13]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.decomposition import PCA
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import time

In [14]:
cleaned_defenders_df.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,...,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,DC,3,17,20,0,0,60,14,0,0,...,16,4,1.123377,0.181910,0.622351,0.333472,0.0,90.0,0.490429,90.000000
3,DL,2,11,12,0,0,19,3,13,4,...,3,3,1.214286,0.232282,0.729104,0.345320,0.0,90.0,0.310037,74.000000
6,DC,6,20,101,0,15,75,11,0,0,...,21,4,1.222135,0.145895,0.647770,0.362843,0.0,90.0,2.182100,87.428571
9,DC,3,4,3,0,0,30,3,0,0,...,12,3,1.175325,0.202411,0.640516,0.380556,0.0,90.0,0.761140,90.000000
12,DR,3,19,24,0,4,30,0,0,20,...,20,4,1.207792,0.206028,0.666690,0.313085,0.0,90.0,1.615328,90.000000


In [15]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
cleaned_defenders_df.describe()

,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,goals,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tackles,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
count,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000,653.000000
mean,9.032159,35.251149,57.344564,0.839204,6.425727,58.500766,11.836141,11.013783,20.964778,2.393568,8.030628,4.108729,12.565084,21.024502,0.869832,65.952527,62.292496,62.690658,3.739663,87.826953,65.808576,0.030628,0.889740,0.188361,887.574273,142.378254,242.978560,0.078101,6.924962,2.807044,2.117917,11.598775,38.396631,2.799387,7.528331,1434.096478,16.814701,3.664625,1.234124,0.189594,0.672989,0.397633,0.004536,90.708591,1.015565,87.764537
std,9.493001,37.631382,76.587769,2.396993,9.324061,63.838601,14.052021,21.037174,36.317157,3.539404,11.952833,7.399561,14.759861,22.798719,2.067843,64.435890,62.722575,63.387351,7.645094,104.258935,74.766190,0.361887,2.083407,0.821333,962.380508,147.044073,233.264502,0.437885,8.872419,4.771093,3.369368,15.937901,40.429828,5.220406,11.189029,1424.648869,18.424634,4.741727,0.164887,0.042920,0.054191,0.083284,0.007867,2.862990,0.475800,7.233892
min,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,6.000000,0.000000,2.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.000000,0.000000,0.000000,0.533333,0.052705,0.469444,0.114286,0.000000,90.000000,0.000000,28.000000
25%,2.000000,8.000000,10.000000,0.000000,0.000000,14.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,3.000000,4.000000,0.000000,18.000000,16.000000,16.000000,0.000000,20.000000,14.000000,0.000000,0.000000,0.000000,232.000000,36.000000,72.000000,0.000000,0.000000,0.000000,0.000000,0.000000,10.000000,0.000000,0.000000,399.000000,4.000000,0.000000,1.140788,0.166890,0.637790,0.346667,0.000000,90.000000,0.718306,87.043478
50%,4.000000,24.000000,28.000000,0.000000,4.000000,36.000000,8.000000,3.000000,8.000000,0.000000,4.000000,0.000000,8.000000,14.000000,0.000000,41.000000,40.000000,36.000000,0.000000,48.000000,36.000000,0.000000,0.000000,0.000000,538.000000,82.000000,154.000000,0.000000,4.000000,0.000000,0.000000,6.000000,24.000000,0.000000,4.000000,899.000000,10.000000,3.000000,1.229020,0.187991,0.670655,0.394748,0.000000,90.000000,0.958255,90.000000
75%,14.000000,48.000000,69.000000,0.000000,8.000000,80.000000,18.000000,12.000000,22.000000,4.000000,10.000000,4.000000,18.000000,30.000000,0.000000,102.000000,88.000000,94.000000,4.000000,112.000000,95.000000,0.000000,0.000000,0.000000,1192.000000,204.000000,352.000000,0.000000,10.000000,4.000000,3.000000,16.000000,54.000000,4.000000,10.000000,2015.000000,23.000000,6.000000,1.352715,0.213453,0.708611,0.450000,0.006867,90.000000,1.296944,90.000000
max,38.000000,201.000000,640.000000,31.000000,84.000000,370.000000,86.000000,251.000000,306.000000,27.000000,97.000000,67.000000,89.000000,140.000000,21.000000,350.000000,346.000000,309.000000,96.000000,640.000000,438.000000,6.000000,18.000000,8.000000,5660.000000,692.000000,1173.000

In [16]:
cleaned_defenders_df.columns

Index(['pos_role', 'matches_played', 'aerials_l', 'aerials_w', 'assists',
       'chances2score', 'clearances', 'countattack', 'crosses_acc',
       'crosses_inacc', 'dangmistakes', 'drib_success', 'drib_unsuccess',
       'dribbled_past', 'fouls', 'goals', 'grduels_l', 'grduels_w',
       'interceptions', 'keypasses', 'lballs_acc', 'lballs_inacc',
       'missed_penalties', 'offsides', 'owngoals', 'passes_acc',
       'passes_inacc', 'poss_lost', 'rcards', 'shots_offtarget',
       'shots_ontarget', 'shotsblocked', 'stop_shots', 'tackles', 'tballs_acc',
       'tballs_inacc', 'touches', 'wasfouled', 'ycards', 'degree_centrality',
       'betweenness_centrality', 'closeness_centrality', 'flow_centrality',
       'betweenness2goals', 'game_duration', 'flow_success', 'minutesPlayed'],
      dtype='object')

In [17]:
cleaned_defenders_df.tackles.value_counts()

tackles
12     41
8      38
4      36
0      36
16     35
20     25
24     24
3      20
28     16
6      16
36     11
48     11
32      9
22      9
9       9
52      9
21      9
15      8
68      8
42      8
14      8
40      8
62      7
10      7
44      7
56      6
34      6
18      6
58      6
19      5
90      5
26      5
123     5
86      5
35      5
39      5
30      5
66      4
98      4
74      4
23      4
110     4
70      4
94      4
82      4
17      4
57      4
2       4
104     3
100     3
49      3
50      3
60      3
78      3
38      3
33      3
27      3
25      3
31      3
46      3
113     2
116     2
41      2
69      2
173     2
65      2
120     2
59      2
126     2
102     2
84      2
89      2
37      2
107     2
55      2
47      2
51      2
54      2
11      2
67      2
108     2
76      2
156     2
88      2
149     2
132     2
61      1
208     1
148     1
151     1
63      1
133     1
87      1
206     1
73      1
101     1
115     1
211     1
137     1
81

In [18]:
# lets move towards splitting the data.

# get the locations
y = cleaned_defenders_df['tackles']
X = cleaned_defenders_df.drop(columns=['tackles'], axis=1)

In [19]:
X.head()

,pos_role,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,goals,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed
0,DC,3,17,20,0,0,60,14,0,0,4,4,0,4,16,0,22,44,33,0,59,31,0,3,0,344,59,105,0,0,0,0,0,0,0,542,16,4,1.123377,0.181910,0.622351,0.333472,0.0,90.0,0.490429,90.000000
3,DL,2,11,12,0,0,19,3,13,4,0,0,0,0,7,0,13,18,14,0,43,35,0,0,0,133,65,107,0,3,0,3,0,0,0,380,3,3,1.214286,0.232282,0.729104,0.345320,0.0,90.0,0.310037,74.000000
6,DC,6,20,101,0,15,75,11,0,0,8,0,0,3,4,0,30,41,63,0,63,36,0,0,0,747,68,168,0,15,0,4,15,0,0,1139,21,4,1.222135,0.145895,0.647770,0.362843,0.0,90.0,2.182100,87.428571
9,DC,3,4,3,0,0,30,3,0,0,0,0,0,4,3,0,25,48,10,0,50,19,0,0,0,408,41,90,0,0,0,0,11,0,0,602,12,3,1.175325,0.202411,0.640516,0.380556,0.0,90.0,0.761140,90.000000
12,DR,3,19,24,0,4,30,0,0,20,0,0,3,27,19,0,32,54,12,0,24,7,0,0,0,232,41,118,0,8,0,0,3,0,8,535,20,4,1.207792,0.206028,0.666690,0.313085,0.0,90.0,1.615328,90.000000


In [20]:
y.head()

0     28
3     18
6     33
9     30
12    43
Name: tackles, dtype: int64

In [21]:
X.columns

Index(['pos_role', 'matches_played', 'aerials_l', 'aerials_w', 'assists',
       'chances2score', 'clearances', 'countattack', 'crosses_acc',
       'crosses_inacc', 'dangmistakes', 'drib_success', 'drib_unsuccess',
       'dribbled_past', 'fouls', 'goals', 'grduels_l', 'grduels_w',
       'interceptions', 'keypasses', 'lballs_acc', 'lballs_inacc',
       'missed_penalties', 'offsides', 'owngoals', 'passes_acc',
       'passes_inacc', 'poss_lost', 'rcards', 'shots_offtarget',
       'shots_ontarget', 'shotsblocked', 'stop_shots', 'tballs_acc',
       'tballs_inacc', 'touches', 'wasfouled', 'ycards', 'degree_centrality',
       'betweenness_centrality', 'closeness_centrality', 'flow_centrality',
       'betweenness2goals', 'game_duration', 'flow_success', 'minutesPlayed'],
      dtype='object')

In [22]:
X.dtypes

pos_role                   object
matches_played              int64
aerials_l                   int64
aerials_w                   int64
assists                     int64
chances2score               int64
clearances                  int64
countattack                 int64
crosses_acc                 int64
crosses_inacc               int64
dangmistakes                int64
drib_success                int64
drib_unsuccess              int64
dribbled_past               int64
fouls                       int64
goals                       int64
grduels_l                   int64
grduels_w                   int64
interceptions               int64
keypasses                   int64
lballs_acc                  int64
lballs_inacc                int64
missed_penalties            int64
offsides                    int64
owngoals                    int64
passes_acc                  int64
passes_inacc                int64
poss_lost                   int64
rcards                      int64
shots_offtarge

In [23]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

num_features = X.select_dtypes(exclude="object").columns
cat_features = X.select_dtypes(include="object").columns

numeric_transformer = MinMaxScaler()  
oh_transformer = OneHotEncoder()

preprocessor = ColumnTransformer(
    [
        ("OneHotEncoder", oh_transformer, cat_features),
        ("StandardScaler", numeric_transformer, num_features),
    ]
)
X = preprocessor.fit_transform(X)
# Get the numerical column names after scaling
num_col_names = num_features

# Get the categorical column names after one hot encoding
cat_col_names = preprocessor.named_transformers_['OneHotEncoder'].get_feature_names_out(cat_features)

# Combine all column names
all_col_names = list(num_col_names) + list(cat_col_names)
X= pd.DataFrame(X, columns=all_col_names)
print(X.head())

   matches_played  aerials_l  aerials_w   assists  chances2score  clearances  \
0             1.0        0.0        0.0  0.054054       0.084577    0.031250   
1             0.0        1.0        0.0  0.027027       0.054726    0.018750   
2             1.0        0.0        0.0  0.135135       0.099502    0.157813   
3             1.0        0.0        0.0  0.054054       0.019900    0.004688   
4             0.0        0.0        1.0  0.054054       0.094527    0.037500   

   countattack  crosses_acc  crosses_inacc  dangmistakes  drib_success  \
0          0.0     0.000000       0.162162      0.162791      0.000000   
1          0.0     0.000000       0.051351      0.034884      0.051793   
2          0.0     0.178571       0.202703      0.127907      0.000000   
3          0.0     0.000000       0.081081      0.034884      0.000000   
4          0.0     0.047619       0.081081      0.000000      0.000000   

   drib_unsuccess  dribbled_past     fouls     goals  grduels_l  grduels_w

In [24]:
X.shape

(653, 48)

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Now, split the train+validation set into separate training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)  # 0.25 x 0.8 = 0.2

# Display the sizes of the splits to ensure they are correct
print(f"Training set size: {len(X_train)}")
print(f"Validation set size: {len(X_val)}")
print(f"Test set size: {len(X_test)}")

Training set size: 391
Validation set size: 131
Test set size: 131


In [26]:
X_train.head()

,matches_played,aerials_l,aerials_w,assists,chances2score,clearances,countattack,crosses_acc,crosses_inacc,dangmistakes,drib_success,drib_unsuccess,dribbled_past,fouls,goals,grduels_l,grduels_w,interceptions,keypasses,lballs_acc,lballs_inacc,missed_penalties,offsides,owngoals,passes_acc,passes_inacc,poss_lost,rcards,shots_offtarget,shots_ontarget,shotsblocked,stop_shots,tballs_acc,tballs_inacc,touches,wasfouled,ycards,degree_centrality,betweenness_centrality,closeness_centrality,flow_centrality,betweenness2goals,game_duration,flow_success,minutesPlayed,pos_role_DC,pos_role_DL,pos_role_DR
299,1.0,0.0,0.0,0.837838,0.537313,1.000000,0.064516,0.535714,1.000000,0.348837,0.035857,0.045752,1.0,0.340206,0.089552,0.471910,0.735714,0.571429,0.705714,0.812139,1.000000,0.062500,0.943750,0.652968,0.0,0.166667,0.0,0.744252,0.823699,0.806149,0.00,0.716667,0.528302,0.500000,0.758929,0.115385,0.17,0.977951,0.609023,0.514286,0.687015,0.546521,0.496420,0.500988,0.151799,0.0,0.312935,0.674630
112,1.0,0.0,0.0,0.027027,0.059701,0.089063,0.000000,0.107143,0.032432,0.069767,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.064286,0.000000,0.042857,0.043353,0.029126,0.000000,0.028125,0.041096,0.0,0.000000,0.0,0.018040,0.065029,0.057216,0.75,0.150000,0.000000,0.000000,0.026786,0.000000,0.00,0.035426,0.022556,0.000000,0.462624,0.299025,0.365993,0.368814,0.000000,0.0,0.399281,0.673913
532,1.0,0.0,0.0,0.135135,0.119403,0.068750,0.000000,0.000000,0.237838,0.279070,0.000000,0.000000,0.0,0.123711,0.000000,0.134831,0.171429,0.190476,0.125714,0.127168,0.194175,0.041667,0.150000,0.073059,0.0,0.000000,0.0,0.136187,0.104046,0.145175,0.00,0.066667,0.075472,0.166667,0.071429,0.153846,0.04,0.160811,0.240602,0.000000,0.548697,0.468474,0.350541,0.432456,0.192308,0.0,0.385185,0.673913
462,0.0,0.0,1.0,0.027027,0.009950,0.003125,0.000000,0.000000,0.010811,0.046512,0.015936,0.065359,0.0,0.103093,0.029851,0.044944,0.057143,0.000000,0.051429,0.069364,0.045307,0.020833,0.034375,0.036530,0.0,0.000000,0.0,0.021931,0.054913,0.063194,0.50,0.033333,0.000000,0.083333,0.000000,0.115385,0.04,0.037924,0.075188,0.000000,0.604839,0.548684,0.520058,0.617489,0.000000,0.0,0.154175,0.673913
613,0.0,0.0,1.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.046512,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.057143,0.000000,0.011429,0.023121,0.000000,0.000000,0.000000,0.009132,0.0,0.000000,0.0,0.006013,0.011561,0.008540,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.006762,0.030075,0.114286,0.153226,0.177668,0.341463,0.049494,0.000000,0.0,0.000000,0.673913


In [31]:
# Save each DataFrame as a CSV file
X.to_csv('defenders_rating_saves_X.csv', index=False)
y.to_csv('defenders_rating_saves_y.csv', index=False)


print("DataFrames have been saved as CSV files.")

DataFrames have been saved as CSV files.
